In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import pymongo

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw
from climatools.cliradlw.utils import mapband_new2old

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *

import IPython.display as display

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def make_query(param=None):
    return {'param.' + name: value for name, value in param.items()}   

In [4]:
def show_html(s): display.display(display.HTML(s))
    
def show_markdown(s): display.display(display.Markdown(s))

In [5]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [6]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    dims = ['igg', 'g', 'i']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')     
    return ds['coolrg']

def pltdata_cooling_1():
    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(DS_COOL.sel(i=1)),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(DS_COOL_CRD.sum('g')),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(DS_COOL_WGT.sel(igg=10).sum('g')),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data

def nice_xlims(pltdata=None, prange=None):
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)

def plt_cool(pltdata=None):
    plin = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='linear',
                                  prange=(50, 1050))
    plin.plot_width, plin.plot_height = 400, 500
    plog = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='log',
                                  prange=(.01, 200))
    plog.plot_width, plog.plot_height = 400, 500
    everything = gridplot(plin, plog, ncols=2)
    return everything
    
def script_plt_cooling():
    pltdata = pltdata_cooling_1()
    p = plt_cool(pltdata=pltdata)
    show(p)
    show_markdown('''*FIGURE.* Cooling rate profiles.''')

In [7]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels: ds = ds.isel(pressure=ilevels)

    dims = ['i', 'igg', 'g']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df

def tbdata_flux():
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    atmpro = PARAM['atmpro']
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(DS_FLUX_CRD.sum('g'), ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10',
            'df': fmt_flux(DS_FLUX_WGT.sum('g').sel(igg=10), 
                           ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(DS_FLUX.sel(i=1), ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data

def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        show_markdown(f'''*TABLE.*  Fluxes. {d['label']}''')
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        show_markdown(
            f'''*TABLE.*  Fluxes.  ({d['label']}) - ({d0['label']})''')
        
    show_tb(benchmark)
    for d in others: show_tb(d)
    for d in others: show_tbdiff(d=d, d0=benchmark)
        
def script_tb_flux():
    data = tbdata_flux()
    tb_flux(tbdata=data)

In [8]:
def gasbands():
    bmap = mapband_new2old()
    gases = [gas for gas, _ in PARAM['molecule'].items()] 
    bands = [bmap[b] for b in PARAM['band']]
    return list(itertools.product(gases, bands))

def print_bestfit_params():
    df = pd.DataFrame()
    for gas, band in gasbands():
        param = bestfits.kdist_params(molecule=gas, band=band)
        srs = pd.Series(param)
        df[f'({gas}, {band})'] = srs
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df.sort_index())
    pd.reset_option('display.max_colwidth')
        
def print_input_param():
    df_clirad = pd.Series(PARAM, name='clirad run').to_frame()
    df_lblnew = pd.Series(PARAM_LBLNEW, name='lblnew run compared against').to_frame()
    df = pd.merge(df_clirad, df_lblnew, how='outer', 
                  left_index=True, right_index=True)
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df)
    pd.reset_option('display.max_colwidth')
        
def print_lblnew_param():
    pd.set_option('display.max_colwidth', -1)
    display.display(pd.Series(PARAM_LBLNEW).to_frame())
    pd.reset_option('display.max_colwidth')

In [9]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD.sum('g'), DS_COOL.sel(i=1)]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [10]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD.sum('g'), DS_FLUX.sel(i=1)]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [11]:
def script_fluxcoolr_excelfile():
    '''
    Save fluxes and cooling rates by layer by g-interval in excel file
    and make it available for download.
    '''
    fname = 'output_byg.xlsx'
    
    writer = pd.ExcelWriter(fname)
    
    for g in DS_FLUX_WGT.coords['g']:
        df = DS_FLUX_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'flux g={int(g)}')
        
    for g in DS_COOL_WGT.coords['g']:
        df = DS_COOL_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'cool g={int(g)}')    

In [12]:


    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()

In [13]:
%run param.py

client = pymongo.MongoClient('localhost', 27017)

doc = client.lblnew.bestfit_lw.find_one(make_query(param=PARAM_LBLNEW))
DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_fluxg']))
DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolrg']))
DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wfluxg']))
DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolrg']))

doc = client.cliradnew.lw.find_one(make_query(param=PARAM))
DS_FLUX = load_output_file(io.StringIO(doc['output_flux']))
DS_COOL = load_output_file(io.StringIO(doc['output_coolr']))

In [14]:
script()
if 'ng_refs' in PARAM_LBLNEW:
    script_fluxcoolr_excelfile()

### CLIRAD-LW

### Table of Contents

# Input Parameters

,clirad run,lblnew run compared against
atmpro,saw,saw
band,[7],5
commitnumber,1013f91,5014a19
conc,-,-
dv,-,0.001
klin,-,1e-24
molecule,{'h2o': 'atmpro'},h2o
ng_adju,-,[0]
ng_refs,-,[3]
nv,-,1000


# Best-fit Parameters

...

*TABLE.*  Fluxes.  (CLIRAD) - (CRD)

# Cooling Rate by Layer

CRD               CLIRAD `coolr_bands`              
            layer        coolrg                layer        coolrg
pressure                                                          
0.000312        1 -1.739972e-06                    1  3.126796e-05
0.000750        2 -1.691339e-06                    2  9.040285e-05
0.001052        3 -1.547010e-06                    3  9.382467e-06
0.001476        4 -1.321156e-06                    4  6.487670e-06
0.002070        5 -9.952471e-07                    5  4.480552e-06
0.002904        6 -5.475467e-07                    6  3.212723e-06
0.004074        7  4.767115e-08                    7  2.854207e-06
0.005714        8  8.226801e-07                    8  2.234739e-06
0.008015        9  1.812960e-06                    9  2.113059e-06
0.011243       10  3.060598e-06                   10  2.241106e-06
0.015771       11  4.614836e-06                   11  2.607201e-06
0.022122       12  6.531551e-06                   12  3.157446e-06
0.031031       13  8.864591e-06                   13  2.994083e-06
0.043528       14  1.133795e-05                   14  3.560689e-06
0.061057       15  1.354673e-05                   15  5.251457e-06
0.085645       16  1.568813e-05                   16  6.581130e-06
0.120136       17  1.805783e-05                   17  7.600595e-06
0.168516       18  2.068564e-05                   18  8.689492e-06
0.236378       19  2.359606e-05                   19  9.862004e-06
0.331549       20  2.681014e-05                   20  1.104663e-05
0.465100       21  2.998143e-05                   21  1.169673e-05
0.652400       22  2.748034e-05                   22  1.090228e-05
0.915100       23  1.889242e-05                   23  7.868683e-06
1.283650       24  1.125698e-05                   24  4.678840e-06
1.800600       25  6.080012e-06                   25  2.380899e-06
2.525700       26  2.675685e-06                   26  8.635192e-07
3.542800       27  5.348150e-07                   27 -8.854945e-08
4.969550       28 -6.137896e-07                   28 -6.120215e-07
6.970850       29 -1.139138e-06                   29 -8.713299e-07
9.778100       30 -1.427074e-06                   30 -1.017084e-06
13.715850      31 -1.611923e-06                   31 -1.114749e-06
19.239350      32 -1.729567e-06                   32 -1.178823e-06
26.987250      33 -1.759404e-06                   33 -1.205998e-06
37.855300      34 -1.731227e-06                   34 -1.196148e-06
53.100050      35 -1.681306e-06                   35 -1.231692e-06
73.887500      36 -1.619160e-06                   36 -1.272834e-06
97.662500      37 -1.569147e-06                   37 -1.284795e-06
121.437500     38 -1.577798e-06                   38 -1.312573e-06
145.212500     39 -2.219998e-06                   39 -1.833654e-06
168.987500     40 -2.966301e-06                   40 -2.435510e-06
192.762500     41 -3.871196e-06                   41 -3.170145e-06
216.537500     42 -4.634976e-06                   42 -3.851708e-06
240.312500     43 -5.426818e-06                   43 -4.593924e-06
264.087500     44 -6.373958e-06                   44 -5.460716e-06
287.862500     45 -7.432478e-06                   45 -6.418366e-06
311.637500     46 -8.190233e-06                   46 -7.182777e-06
335.412500     47 -7.924950e-06                   47 -7.151114e-06
359.187500     48 -6.067689e-06                   48 -6.040785e-06
382.962500     49 -2.715752e-06                   49 -4.389294e-06
406.737500     50  7.779120e-06                   50  2.104117e-06
430.512500     51  3.238813e-05                   51  1.887120e-05
454.287500     52  7.902770e-05                   52  5.231947e-05
478.062500     53  1.564385e-04                   53  1.104590e-04
501.837500     54  2.732544e-04                   54  2.016784e-04
525.612500     55  4.367586e-04                   55  3.295768e-04
549.387500     56  6.511740e-04                   56  5.021070e-04
573.162500     57  9.174623e-04                   57  7.22838

# Fluxes by Layer

CRD                              CLIRAD  \
                        flug          fldg      fnetg       flug   
pressure    level                                                  
0.000000    1     -15.040846  0.000000e+00 -15.040846 -15.043304   
0.000624    2     -15.040846  1.095316e-10 -15.040846 -15.043304   
0.000876    3     -15.040846  1.579624e-10 -15.040846 -15.043304   
0.001229    4     -15.040846  2.398500e-10 -15.040846 -15.043304   
0.001723    5     -15.040846  3.783968e-10 -15.040846 -15.043304   
0.002417    6     -15.040846  6.122651e-10 -15.040846 -15.043304   
0.003391    7     -15.040846  1.006281e-09 -15.040846 -15.043304   
0.004757    8     -15.040846  1.668696e-09 -15.040846 -15.043304   
0.006672    9     -15.040846  2.779945e-09 -15.040846 -15.043304   
0.009359    10    -15.040846  4.640309e-09 -15.040846 -15.043304   
0.013128    11    -15.040846  7.748377e-09 -15.040846 -15.043304   
0.018415    12    -15.040846  1.293049e-08 -15.040846 -15.043304   
0.025830    13    -15.040846  2.155316e-08 -15.040846 -15.043304   
0.036232    14    -15.040846  3.586128e-08 -15.040846 -15.043304   
0.050823    15    -15.040846  5.910685e-08 -15.040846 -15.043304   
0.071291    16    -15.040846  9.558640e-08 -15.040846 -15.043304   
0.100000    17    -15.040846  1.519189e-07 -15.040846 -15.043304   
0.140271    18    -15.040846  2.388141e-07 -15.040846 -15.043304   
0.196760    19    -15.040846  3.727633e-07 -15.040846 -15.043304   
0.275997    20    -15.040846  5.791010e-07 -15.040846 -15.043304   
0.387100    21    -15.040846  8.965925e-07 -15.040845 -15.043304   
0.543100    22    -15.040846  1.380935e-06 -15.040845 -15.043304   
0.761700    23    -15.040846  2.017127e-06 -15.040844 -15.043304   
1.068500    24    -15.040846  2.700195e-06 -15.040843 -15.043304   
1.498800    25    -15.040846  3.383049e-06 -15.040843 -15.043304   
2.102400    26    -15.040846  4.060061e-06 -15.040842 -15.043304   
2.949000    27    -15.040847  4.725633e-06 -15.040842 -15.043304   
4.136600    28    -15.040847  5.376167e-06 -15.040842 -15.043305   
5.802500    29    -15.040848  6.039746e-06 -15.040842 -15.043305   
8.139200    30    -15.040849  6.781209e-06 -15.040842 -15.043306   
11.417000   31    -15.040851  7.650383e-06 -15.040843 -15.043307   
16.014700   32    -15.040852  8.685508e-06 -15.040844 -15.043308   
22.464000   33    -15.040855  9.945181e-06 -15.040845 -15.043309   
31.510500   34    -15.040858  1.164865e-05 -15.040847 -15.043312   
44.200100   35    -15.040863  1.421099e-05 -15.040849 -15.043315   
62.000000   36    -15.040870  1.815119e-05 -15.040853 -15.043320   
85.775000   37    -15.040881  2.391443e-05 -15.040857 -15.043327   
109.550000  38    -15.040892  3.007958e-05 -15.040861 -15.043335   
133.325000  39    -15.040903  3.646769e-05 -15.040866 -15.043343   
157.100000  40    -15.040918  4.534817e-05 -15.040872 -15.043354   
180.875000  41    -15.040938  5.706903e-05 -15.040881 -15.043369   
204.650000  42    -15.040964  7.216508e-05 -15.040892 -15.043389   
228.425000  43    -15.040995  9.001272e-05 -15.040905 -15.043412   
252.200000  44    -15.041030  1.106585e-04 -15.040920 -15.043441   
275.975000  45    -15.041072  1.346414e-04 -15.040938 -15.043474   
299.750000  46    -15.041122  1.628027e-04 -15.040959 -15.043514   
323.525000  47    -15.041180  1.983866e-04 -15.040982 -15.043561   
347.300000  48    -15.041252  2.480669e-04 -15.041005 -15.043619   
371.075000  49    -15.041341  3.195514e-04 -15.041021 -15.043691   
394.850000  50    -15.041477  4.478362e-04 -15.041029 -15.043801   
418.625000  51    -15.041694  6.871877e-04 -15.041007 -15.043977   
442.400000  52    -15.042019  1.102706e-03 -15.040916 -15.044238   
466.175000  53    -15.042474  1.780138e-03 -15.040694 -15.044604   
489.950000  54    -15.043077  2.824090e-03 -15.040253 -15.045091   
513.725000  55    -15.043838  4.354917e-03 -15.039484 -15.045710   
537.500000  56    -15.044755  6.502316e-03 -15.038253 -15.046450   
561.275000  5

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')